In [1]:
%cd "OneDrive - National University of Singapore"

C:\Users\wongp\OneDrive - National University of Singapore


In [2]:
%cd CS4248/Project/db/

C:\Users\wongp\OneDrive - National University of Singapore\CS4248\Project\db


In [3]:
import os
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from collections import Counter
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import nltk
from nltk.stem import WordNetLemmatizer

Convert training data into csv file

In [ ]:
import os
import pandas as pd

pathNeg = "train/neg"
files = os.listdir(pathNeg)
data = []
for file in files:
  if os.path.isfile(os.path.join(pathNeg, file)):
    id = int(file.split("_")[0])
    with open(os.path.join(pathNeg, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 0])

sizeOfNeg = 12500

pathPos = "train/pos"
files = os.listdir(pathPos)
for file in files:
  if os.path.isfile(os.path.join(pathPos, file)):
    id = int(file.split("_")[0]) + sizeOfNeg
    with open(os.path.join(pathPos, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 1])


df = pd.DataFrame(data, columns = ['id', 'text', 'score']).set_index('id').sort_index()
df.to_csv('train.csv')

Convert testing data into csv file

In [ ]:
pathNeg = "test/neg"
files = os.listdir(pathNeg)
data = []
for file in files:
  if os.path.isfile(os.path.join(pathNeg, file)):
    id = int(file.split("_")[0])
    with open(os.path.join(pathNeg, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 0])

sizeOfNeg = 12500

pathPos = "test/pos"
files = os.listdir(pathPos)
for file in files:
  if os.path.isfile(os.path.join(pathPos, file)):
    id = int(file.split("_")[0]) + sizeOfNeg
    with open(os.path.join(pathPos, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 1])


df = pd.DataFrame(data, columns = ['id', 'text', 'score']).set_index('id').sort_index()
df.to_csv('test.csv')

convert vocab into csv file

In [ ]:
with open("imdb.vocab", encoding='utf8') as f:
  vocab = []
  for line in f:
    vocab.append(line.rstrip())

vocab_df = pd.DataFrame(vocab, columns=['vocab'])
print(vocab_df)
vocab_df.to_csv('vocab_given.csv', index=False)

                   vocab
0                    the
1                    and
2                      a
3                     of
4                     to
...                  ...
89522          copywrite
89523             artbox
89524          kinky-sex
89525           urrrghhh
89526  investigator-like

[89527 rows x 1 columns]


In [4]:
from nltk.tokenize import word_tokenize

def tokenizeReview(review):
  review = review.replace('.', ' . ')
  tokens = word_tokenize(review.lower())
  return tokens

Get vocab from training set

In [ ]:
from collections import Counter
reviews = train_df['text'].tolist()
reviews = list(map(tokenizeReview, reviews))
reviews = [item for review in reviews for item in review]
vdict = Counter(reviews).most_common()

In [ ]:
vocabs = []
for pair in vdict:
  vocabs.append(pair[0])

vocab_df = pd.DataFrame(vocabs, columns=['vocab'])
vocab_df.to_csv('vocab.csv', index=False)
print(vocab_df)

              vocab
0               the
1                 .
2                 ,
3               and
4                 a
...             ...
99203    people-and
99204         padre
99205   unmatchable
99206  scenery-wise
99207  non-teenager

[99208 rows x 1 columns]


Read training data and testing data from csv file (START HERE after initialization)

In [5]:
train_df = pd.read_csv('train/train.csv')
test_df = pd.read_csv('test/test.csv')
vocab_df = pd.read_csv('vocab.csv')
print(vocab_df)

              vocab
0               the
1                 .
2                 ,
3               and
4                 a
...             ...
99203    people-and
99204         padre
99205   unmatchable
99206  scenery-wise
99207  non-teenager

[99208 rows x 1 columns]


In [ ]:
vocab_dict = pd.Series(vocab_df.index + 1, index=vocab_df.vocab).to_dict()
print(vocab_dict)

In [7]:
def convertToIndex(review):
  vec = []
  for word in review:
    if word in vocab_dict:
      vec.append(int(vocab_dict[word]))
    else:
      vec.append(0)
  return vec

In [8]:
train_df['tokens'] = train_df['text'].apply(tokenizeReview)

In [9]:
train_df['vec'] = train_df['tokens'].apply(convertToIndex)

In [ ]:
train_df

,id,text,score,tokens
0,0,Story of a man who has unnatural feelings for ...,0,"[story, of, a, man, who, has, unnatural, feeli..."
1,1,Robert DeNiro plays the most unbelievably inte...,0,"[robert, deniro, plays, the, most, unbelievabl..."
2,2,"I saw the capsule comment said ""great acting.""...",0,"[i, saw, the, capsule, comment, said, ``, grea..."
3,3,If I had not read Pat Barker's 'Union Street' ...,0,"[if, i, had, not, read, pat, barker, 's, 'unio..."
4,4,This fanciful horror flick has Vincent Price p...,0,"[this, fanciful, horror, flick, has, vincent, ..."
...,...,...,...,...
24995,24995,What's Good About It: Some inventive and genui...,1,"[what, 's, good, about, it, :, some, inventive..."
24996,24996,For years we've been watching every horror fil...,1,"[for, years, we, 've, been, watching, every, h..."
24997,24997,If you haven't already seen this movie of Mary...,1,"[if, you, have, n't, already, seen, this, movi..."
24998,24998,this movie is the best movie ever it has a lot...,1,"[this, movie, is, the, best, movie, ever, it, ..."


In [ ]:
print('Maximum review length: {}'.format(
len(max((train_df['tokens']), key=len))))
print('Minimum review length: {}'.format(
len(min((train_df['tokens']), key=len))))

Maximum review length: 2820
Minimum review length: 11


Lemmatizer

In [10]:
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')

def lemmatizer(reviewTokens):
  punctuations="?:!.,;"
  for word in reviewTokens:
    if word in punctuations:
      reviewTokens.remove(word)
  reviewLemm = []
  for word in reviewTokens:
    newWord = wordnet_lemmatizer.lemmatize(word, pos="v")
    reviewLemm.append(newWord)

  return reviewLemm


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wongp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\wongp\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [11]:
train_df['lemmTokens'] = train_df['tokens'].apply(lemmatizer)

In [ ]:
train_df

,id,text,score,tokens,lemmTokens
0,0,Story of a man who has unnatural feelings for ...,0,"[story, of, a, man, who, has, unnatural, feeli...","[story, of, a, man, who, have, unnatural, feel..."
1,1,Robert DeNiro plays the most unbelievably inte...,0,"[robert, deniro, plays, the, most, unbelievabl...","[robert, deniro, play, the, most, unbelievably..."
2,2,"I saw the capsule comment said ""great acting.""...",0,"[i, saw, the, capsule, comment, said, ``, grea...","[i, saw, the, capsule, comment, say, ``, great..."
3,3,If I had not read Pat Barker's 'Union Street' ...,0,"[if, i, had, not, read, pat, barker, 's, 'unio...","[if, i, have, not, read, pat, barker, 's, 'uni..."
4,4,This fanciful horror flick has Vincent Price p...,0,"[this, fanciful, horror, flick, has, vincent, ...","[this, fanciful, horror, flick, have, vincent,..."
...,...,...,...,...,...
24995,24995,What's Good About It: Some inventive and genui...,1,"[what, 's, good, about, it, some, inventive, a...","[what, 's, good, about, it, some, inventive, a..."
24996,24996,For years we've been watching every horror fil...,1,"[for, years, we, 've, been, watching, every, h...","[for, years, we, 've, be, watch, every, horror..."
24997,24997,If you haven't already seen this movie of Mary...,1,"[if, you, have, n't, already, seen, this, movi...","[if, you, have, n't, already, see, this, movie..."
24998,24998,this movie is the best movie ever it has a lot...,1,"[this, movie, is, the, best, movie, ever, it, ...","[this, movie, be, the, best, movie, ever, it, ..."


In [12]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words())
stop_words = stop_words.union(["br", "<", ">", "(", ")", "/", "''", "'s", "``", "'", "'ve", "'m"])
def remove_stopwords(review):
  tokens_without_sw = [word for word in review if not word in stop_words]

  return tokens_without_sw

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wongp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
train_df['lemmTokens'] = train_df['lemmTokens'].apply(remove_stopwords)

In [14]:
train_df

,id,text,score,tokens,vec,lemmTokens
0,0,Story of a man who has unnatural feelings for ...,0,"[story, of, a, man, who, has, unnatural, feeli...","[76, 6, 5, 142, 46, 56, 7518, 1387, 21, 5, 451...","[story, unnatural, feel, pig, start, open, sce..."
1,1,Robert DeNiro plays the most unbelievably inte...,0,"[robert, deniro, plays, the, most, unbelievabl...","[649, 5006, 307, 1, 106, 3769, 1076, 9203, 6, ...","[robert, deniro, play, unbelievably, intellige..."
2,2,"I saw the capsule comment said ""great acting.""...",0,"[i, saw, the, capsule, comment, said, ``, grea...","[15, 226, 1, 9586, 906, 311, 28, 102, 129, 2, ...","[saw, capsule, comment, say, great, act, opini..."
3,3,If I had not read Pat Barker's 'Union Street' ...,0,"[if, i, had, not, read, pat, barker, 's, 'unio...","[57, 15, 78, 32, 338, 3189, 7720, 18, 34296, 8...","[read, pat, barker, 'union, street, see, film,..."
4,4,This fanciful horror flick has Vincent Price p...,0,"[this, fanciful, horror, flick, has, vincent, ...","[16, 16016, 213, 512, 56, 3154, 1789, 397, 5, ...","[fanciful, horror, flick, vincent, price, play..."
...,...,...,...,...,...,...
24995,24995,What's Good About It: Some inventive and genui...,1,"[what, 's, good, about, it, some, inventive, a...","[60, 18, 62, 55, 13, 87, 61, 4388, 4, 2028, 93...","[good, inventive, genuinely, creepy, little, e..."
24996,24996,For years we've been watching every horror fil...,1,"[for, years, we, 've, been, watching, every, h...","[21, 168, 82, 155, 93, 163, 188, 213, 25, 17, ...","[years, watch, every, horror, film, dull, holl..."
24997,24997,If you haven't already seen this movie of Mary...,1,"[if, you, have, n't, already, seen, this, movi...","[57, 29, 37, 31, 466, 126, 16, 23, 6, 13586, 4...","[n't, already, see, movie, mary-kate, ashley, ..."
24998,24998,this movie is the best movie ever it has a lot...,1,"[this, movie, is, the, best, movie, ever, it, ...","[16, 23, 8, 1, 133, 23, 138, 13, 56, 5, 186, 6...","[movie, best, movie, ever, lot, live, action, ..."


In [ ]:
from collections import Counter
reviewsLem = train_df['lemmTokens'].tolist()
# reviewsLem = list(map(tokenizeReview, reviewsLem))
reviewsLem = [item for review in reviewsLem for item in review]
vdictLem = Counter(reviewsLem).most_common()
#print(vdictLem)

vdictLemToIdx = {}
idx = 1
for pair in vdictLem:
  vdictLemToIdx[pair[0]] = idx
  idx = idx + 1
print(vdictLemToIdx)

In [16]:
def convertToIndex(review):
  vec = []
  for word in review:
    if word in vdictLemToIdx:
      vec.append(int(vdictLemToIdx[word]))
    else:
      vec.append(0)
  return vec

In [17]:
train_df['vecLem'] = train_df['lemmTokens'].apply(convertToIndex)

In [18]:
train_df

,id,text,score,tokens,vec,lemmTokens,vecLem
0,0,Story of a man who has unnatural feelings for ...,0,"[story, of, a, man, who, has, unnatural, feeli...","[76, 6, 5, 142, 46, 56, 7518, 1387, 21, 5, 451...","[story, unnatural, feel, pig, start, open, sce...","[17, 6245, 53, 2799, 79, 230, 52, 1084, 359, 1..."
1,1,Robert DeNiro plays the most unbelievably inte...,0,"[robert, deniro, plays, the, most, unbelievabl...","[649, 5006, 307, 1, 106, 3769, 1076, 9203, 6, ...","[robert, deniro, play, unbelievably, intellige...","[525, 4212, 33, 3186, 907, 7613, 9, 2, 18980, ..."
2,2,"I saw the capsule comment said ""great acting.""...",0,"[i, saw, the, capsule, comment, said, ``, grea...","[15, 226, 1, 9586, 906, 311, 28, 102, 129, 2, ...","[saw, capsule, comment, say, great, act, opini...","[121, 7763, 298, 24, 29, 32, 519, 42, 29, 67, ..."
3,3,If I had not read Pat Barker's 'Union Street' ...,0,"[if, i, had, not, read, pat, barker, 's, 'unio...","[57, 15, 78, 32, 338, 3189, 7720, 18, 34296, 8...","[read, pat, barker, 'union, street, see, film,...","[158, 2574, 6406, 27981, 747, 6, 1, 14, 5, 434..."
4,4,This fanciful horror flick has Vincent Price p...,0,"[this, fanciful, horror, flick, has, vincent, ...","[16, 16016, 213, 512, 56, 3154, 1789, 397, 5, ...","[fanciful, horror, flick, vincent, price, play...","[13144, 108, 301, 2683, 1393, 33, 987, 5298, 3..."
...,...,...,...,...,...,...,...
24995,24995,What's Good About It: Some inventive and genui...,1,"[what, 's, good, about, it, some, inventive, a...","[60, 18, 62, 55, 13, 87, 61, 4388, 4, 2028, 93...","[good, inventive, genuinely, creepy, little, e...","[10, 3673, 1709, 785, 45, 149, 7, 1824, 15, 46..."
24996,24996,For years we've been watching every horror fil...,1,"[for, years, we, 've, been, watching, every, h...","[21, 168, 82, 155, 93, 163, 188, 213, 25, 17, ...","[years, watch, every, horror, film, dull, holl...","[66, 12, 84, 108, 1, 615, 252, 8870, 5, 121, 1..."
24997,24997,If you haven't already seen this movie of Mary...,1,"[if, you, have, n't, already, seen, this, movi...","[57, 29, 37, 31, 466, 126, 16, 23, 6, 13586, 4...","[n't, already, see, movie, mary-kate, ashley, ...","[3, 358, 6, 2, 11172, 3954, 24, 351, 156, 69, ..."
24998,24998,this movie is the best movie ever it has a lot...,1,"[this, movie, is, the, best, movie, ever, it, ...","[16, 23, 8, 1, 133, 23, 138, 13, 56, 5, 186, 6...","[movie, best, movie, ever, lot, live, action, ...","[2, 46, 2, 49, 61, 73, 104, 29, 192, 12, 184, ..."


In [19]:
max_words = 500
dataset = sequence.pad_sequences(train_df['vecLem'], maxlen=max_words)
y = train_df['score']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(dataset, y, test_size=0.3, stratify=y, random_state = 42)


In [20]:
from gensim.models import Word2Vec
import gensim.downloader as api

v2w_model = api.load('glove-wiki-gigaword-50')
print(v2w_model['computer'])

[ 0.079084 -0.81504   1.7901    0.91653   0.10797  -0.55628  -0.84427
 -1.4951    0.13418   0.63627   0.35146   0.25813  -0.55029   0.51056
  0.37409   0.12092  -1.6166    0.83653   0.14202  -0.52348   0.73453
  0.12207  -0.49079   0.32533   0.45306  -1.585    -0.63848  -1.0053
  0.10454  -0.42984   3.181    -0.62187   0.16819  -1.0139    0.064058
  0.57844  -0.4556    0.73783   0.37203  -0.57722   0.66441   0.055129
  0.037891  1.3275    0.30991   0.50697   1.2357    0.1274   -0.11434
  0.20709 ]


In [21]:
embedding_dim = 100
vocab_len = len(vdictLemToIdx) + 1
embed_vector_len = v2w_model['computer'].shape[0]
emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, i in vdictLemToIdx.items():
  if word in v2w_model:
    embedding_vector = v2w_model.get_vector(word)
    emb_matrix[i] = embedding_vector


In [ ]:
emb_matrix[1]

array([ 0.06912   ,  0.22159   , -0.66613001, -0.14060999,  0.063365  ,
        0.41747001, -0.21723001, -1.04009998,  0.026069  ,  1.33819997,
       -0.083642  ,  0.66324002, -0.45989001,  0.95883   ,  1.31850004,
       -0.62523001,  0.56152999,  0.65288001, -1.23640001,  0.29692   ,
        1.23800004,  0.80620998,  0.40967   ,  0.39945   ,  0.57982999,
       -1.36580002, -1.67639995,  0.069718  , -0.59634   , -0.63116997,
        2.58940005, -0.56914997,  0.46878999, -0.9249    , -0.44628999,
        0.42282999,  0.17636999, -0.014459  , -0.75357997, -0.97850001,
        0.023075  ,  1.2227    , -0.17517   , -1.57430005, -0.95349997,
       -0.10959   ,  0.24517   , -0.93550998, -0.20942999,  0.67333001])

In [ ]:
print(vocab_len)

87517


In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=max_words, weights=[emb_matrix], trainable=True)

embedding_size=32
model=Sequential()
model.add(embedding_layer)
model.add(GRU(units=50, return_sequences = True, input_shape=(X_train.shape[1],1), activation='tanh'))
#model.add(LSTM(100))
model.add(Dropout(0.6))
model.add(GRU(units=50, input_shape=(X_train.shape[1],1), activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 50)           4375850   
                                                                 
 gru_2 (GRU)                 (None, 500, 50)           15300     
                                                                 
 dropout_1 (Dropout)         (None, 500, 50)           0         
                                                                 
 gru_3 (GRU)                 (None, 50)                15300     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 4,406,501
Trainable params: 4,406,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
547/547 [==============================] - 141s 253ms/step - loss: 0.4894 - accuracy: 0.7496
Epoch 2/5
547/547 [==============================] - 143s 262ms/step - loss: 0.2554 - accuracy: 0.9007
Epoch 3/5
547/547 [==============================] - 144s 263ms/step - loss: 0.1240 - accuracy: 0.9583
Epoch 4/5
547/547 [==============================] - 141s 259ms/step - loss: 0.0509 - accuracy: 0.9848
Epoch 5/5
547/547 [==============================] - 142s 260ms/step - loss: 0.0177 - accuracy: 0.9958


In [ ]:
scores = model.evaluate(X_val, y_val, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.868399977684021


In [ ]:
model.save('basic_model')

INFO:tensorflow:Assets written to: basic_gru_model\assets


INFO:tensorflow:Assets written to: basic_gru_model\assets


**Without Glove word embedding**

In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU
embedding_size=32
model=Sequential()
model.add(Embedding(vocab_len, embedding_size, input_length=max_words))
model.add(GRU(units=50, input_shape=(X_train.shape[1],1), activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 32)           2800544   
                                                                 
 gru_4 (GRU)                 (None, 50)                12600     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 2,813,195
Trainable params: 2,813,195
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
547/547 [==============================] - 73s 131ms/step - loss: 0.4104 - accuracy: 0.8010
Epoch 2/5
547/547 [==============================] - 69s 126ms/step - loss: 0.1568 - accuracy: 0.9439
Epoch 3/5
547/547 [==============================] - 69s 125ms/step - loss: 0.0664 - accuracy: 0.9791
Epoch 4/5
547/547 [==============================] - 71s 130ms/step - loss: 0.0339 - accuracy: 0.9890
Epoch 5/5
547/547 [==============================] - 69s 126ms/step - loss: 0.0208 - accuracy: 0.9939


In [ ]:
scores = model.evaluate(X_val, y_val, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.8744000196456909


In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=max_words, weights=[emb_matrix], trainable=True)

embedding_size=32
model=Sequential()
model.add(embedding_layer)
model.add(GRU(units=50, input_shape=(X_train.shape[1],1), activation='tanh'))
#model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

scores = model.evaluate(X_val, y_val, verbose=0)
print('Test accuracy:', scores[1])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 50)           4375850   
                                                                 
 gru_5 (GRU)                 (None, 50)                15300     
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 4,391,201
Trainable params: 4,391,201
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
547/547 [==============================] - 76s 136ms/step - loss: 0.4717 - accuracy: 0.7667
Epoch 2/5
547/547 [==============================] - 75s 138ms/step - loss: 0.2665 - accuracy: 0.8949
Epoch 3/5
547/547 [==============================] - 73s 134ms/step - loss: 0.1492 - accuracy: 0.94

# Test Data

In [30]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=max_words, weights=[emb_matrix], trainable=True)

embedding_size=32
model=Sequential()
model.add(embedding_layer)
model.add(GRU(units=50, return_sequences = True, input_shape=(dataset.shape[1],1), activation='tanh'))
#model.add(LSTM(100))
model.add(Dropout(0.6))
model.add(GRU(units=50, input_shape=(dataset.shape[1],1), activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 50)           4375850   
                                                                 
 gru_4 (GRU)                 (None, 500, 50)           15300     
                                                                 
 dropout_2 (Dropout)         (None, 500, 50)           0         
                                                                 
 gru_5 (GRU)                 (None, 50)                15300     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 4,406,501
Trainable params: 4,406,501
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
batch_size=32
model.fit(dataset, y, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
782/782 [==============================] - 203s 256ms/step - loss: 0.4378 - accuracy: 0.7868
Epoch 2/5
782/782 [==============================] - 195s 249ms/step - loss: 0.2188 - accuracy: 0.9164
Epoch 3/5
782/782 [==============================] - 196s 251ms/step - loss: 0.1082 - accuracy: 0.9639
Epoch 4/5
782/782 [==============================] - 194s 248ms/step - loss: 0.0439 - accuracy: 0.9862
Epoch 5/5
782/782 [==============================] - 198s 254ms/step - loss: 0.0191 - accuracy: 0.9950


In [32]:
test_df['tokens'] = test_df['text'].apply(tokenizeReview)
test_df['lemmTokens'] = test_df['tokens'].apply(lemmatizer)
test_df['lemmTokens'] = test_df['lemmTokens'].apply(remove_stopwords)
test_df['vecLem'] = test_df['lemmTokens'].apply(convertToIndex)
test_x = sequence.pad_sequences(test_df['vecLem'], maxlen=max_words)
test_x

array([[   0,    0,    0, ...,  117,   87,  457],
       [   0,    0,    0, ...,    7,  183,  188],
       [   0,    0,    0, ..., 6450,  310,    8],
       ...,
       [   0,    0,    0, ...,  569,  640, 2134],
       [   0,    0,    0, ...,   15,  277,    9],
       [   0,    0,    0, ..., 2601, 2014,    8]])

In [33]:
test_y = test_df['score']
scores = model.evaluate(test_x, test_y, verbose=0)
print('Test accuracy:', scores[1])

pred = model.predict(test_x)

from sklearn.metrics import accuracy_score
pred_labels = []
for i in pred:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(test_y,pred_labels))

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(test_y,pred_labels)
print(cm)

test_df['pred'] = pred_labels
test_df
test_df.to_csv("test_df_pred_lemma_stopword_trainable_true_gru.csv")

Test accuracy: 0.8496400117874146
Accuracy of prediction on test set :  0.84964
[[10774  1726]
 [ 2033 10467]]


trainable false

In [34]:
embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=max_words, weights=[emb_matrix], trainable=False)

embedding_size=32
model=Sequential()
model.add(embedding_layer)
model.add(GRU(units=50, return_sequences = True, input_shape=(dataset.shape[1],1), activation='tanh'))
#model.add(LSTM(100))
model.add(Dropout(0.6))
model.add(GRU(units=50, input_shape=(dataset.shape[1],1), activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 50)           4375850   
                                                                 
 gru_6 (GRU)                 (None, 500, 50)           15300     
                                                                 
 dropout_3 (Dropout)         (None, 500, 50)           0         
                                                                 
 gru_7 (GRU)                 (None, 50)                15300     
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 4,406,501
Trainable params: 30,651
Non-trainable params: 4,375,850
_________________________________________________________________
None


In [35]:
batch_size=32
model.fit(dataset, y, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
782/782 [==============================] - 182s 230ms/step - loss: 0.5256 - accuracy: 0.7346
Epoch 2/5
782/782 [==============================] - 182s 233ms/step - loss: 0.4015 - accuracy: 0.8214
Epoch 3/5
782/782 [==============================] - 183s 234ms/step - loss: 0.3612 - accuracy: 0.8442
Epoch 4/5
782/782 [==============================] - 179s 229ms/step - loss: 0.3396 - accuracy: 0.8549
Epoch 5/5
782/782 [==============================] - 180s 230ms/step - loss: 0.3214 - accuracy: 0.8623


In [36]:
test_y = test_df['score']
scores = model.evaluate(test_x, test_y, verbose=0)
print('Test accuracy:', scores[1])

pred = model.predict(test_x)

from sklearn.metrics import accuracy_score
pred_labels = []
for i in pred:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(test_y,pred_labels))

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(test_y,pred_labels)
print(cm)

test_df['pred'] = pred_labels
test_df
test_df.to_csv("test_df_pred_lemma_stopword_trainable_false_gru.csv")

Test accuracy: 0.8560000061988831
Accuracy of prediction on test set :  0.856
[[11143  1357]
 [ 2243 10257]]
